### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [46]:
# Importing libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [47]:
engine = create_engine('sqlite:///../DataBase.db') # nome do arquivo

df = pd.read_sql_table('merged_df.db',engine) # nome da tabela
df = df[df['related']<= 1]

x_input = 'message'

X = df[x_input]
Y = df.iloc[:,4:40]

In [48]:
df['genre'].unique()

array(['direct', 'social', 'news'], dtype=object)

### 2. Write a tokenization function to process your text data

In [49]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords  


nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


def tokenize(text):

    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

        
    lemmatizer = WordNetLemmatizer()

    tokens = nltk.wordpunct_tokenize(text)
    text = nltk.Text(tokens)

    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in text if w.isalpha()]
    filtered_tokens = [w for w in clean_tokens if not w in stop_words]  


    return filtered_tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Windows10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Windows10\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [50]:
# message example
df.iloc[8,1]

'I would like to receive the messages, thank you'

In [51]:
tokenize(df.iloc[8,1])

['would', 'like', 'receive', 'message', 'thank']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [52]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [53]:

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
# y atre eache topic related
# X is the message

In [55]:
%%time
pipeline.fit(X_train,y_train)

Wall time: 4.01 s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

In [56]:
%%time

y_pred = pipeline.predict(X_test)

Wall time: 1min 39s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [57]:
from sklearn.metrics import classification_report


In [58]:
print(y_test.columns.to_list())

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [59]:
target_names = y_test.columns.to_list()
for i,col in enumerate(y_test.columns):
    print(i,col)
    print(classification_report(y_test[col], y_pred[:,i]))

0 related
             precision    recall  f1-score   support

          0       0.58      0.46      0.51      1569
          1       0.84      0.89      0.87      4938

avg / total       0.78      0.79      0.78      6507

1 request
             precision    recall  f1-score   support

          0       0.91      0.94      0.92      5403
          1       0.64      0.54      0.59      1104

avg / total       0.86      0.87      0.87      6507

2 offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6481
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6507

3 aid_related
             precision    recall  f1-score   support

          0       0.72      0.79      0.75      3793
          1       0.66      0.56      0.61      2714

avg / total       0.69      0.69      0.69      6507

4 medical_help
             precision    recall  f1-score   support

          0       0.93      0

In [60]:
len(y_pred[:,0])

6507

In [61]:
y_test[col]

21046    0
12461    0
25670    0
13912    0
14719    0
        ..
5004     0
10443    0
4542     1
20668    0
5170     1
Name: direct_report, Length: 6507, dtype: int64

### 6. Improve your model
Use grid search to find better parameters. 

In [62]:
%%time

from sklearn.model_selection import GridSearchCV
pipeline.get_params()

Wall time: 0 ns


{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x000002A6846F3C80>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
              metric_params=None, n_jobs=1, n_neighbors=5, p=2,
              weights='uniform'),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_

In [63]:
%%time

# parameters = {
#     'vect__max_df': (0.5, 0.75, 1.0),
#     'vect__max_features': (None, 3000, 5000),
#     'vect__ngram_range': ((1,1),(1,2)) ,
#     'tfidf__use_idf': (True,False),
#     'clf__estimator__n_neighbors': (3,5,11,15),
#     'clf__estimator__weights': ('uniform','distance'),
#     'clf__estimator__metric': ('minkowski','euclidean','manhattan')

# }

parameters = {
    # 'vect__max_df': (0.5, 0.75),
    'vect__max_features': (1000, 3000),
    # 'vect__ngram_range': ((1,1),(1,2)) ,
    # 'tfidf__use_idf': (True,False),
    'clf__estimator__n_neighbors': (3,5),
    # 'clf__estimator__weights': ('uniform','distance'),
    # 'clf__estimator__metric': ('minkowski','euclidean')

}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4)
cv_results = cv.fit(X_train,y_train)
print(cv.best_params_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
pipeline_tuned = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, max_features= 1000)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier(n_neighbors=3, metric='minkowski')))
])


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [31]:
import pickle
pipeline_tuned.fit(X_train,y_train)

filename = 'finalized_model.sav'
pickle.dump(pipeline_tuned, open(filename, 'wb'))
 
# some time later...
 

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [33]:
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.25328043942630457
